In [1]:
import pandas as pd 
import numpy as np
import re

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
def create_common_df(id_):
    try:
        cols = ['NaturalLanguageID', 'ProgrammingLanguageName', 'RepoID', 'SourceID', 'CommentID', 'comment', 'label']
        df = pd.read_csv('Data\\Common\\OutDataCommon{}.txt'.format(id_), sep='\t', lineterminator='\r', header=None)
        df.columns = cols
        df = df.drop(['NaturalLanguageID', 'ProgrammingLanguageName', 'RepoID', 'SourceID', 'CommentID'],axis=1)
        return df
    except:
        with open('Data\\Common\OutDataCommon{}.txt'.format(id_), "r", encoding="utf-8",  newline="\r\n") as source:
            for line in source:
                line = line.replace("\n", "")
                part = line.split("\t")
                if len(part) != 7:
                    print('POPRAVITI .TXT')
                    print(part)

In [4]:
df1 = create_common_df(1)
df2 = create_common_df(2)
df3 = create_common_df(3)
df4 = create_common_df(4)

In [5]:
df4.shape

(557, 2)

In [6]:
pd.set_option('display.max_rows', 10)

In [7]:
merged = df1.merge(df2, how='left', on='comment', suffixes=('_1', '_2')).merge(df3, how='left', on='comment').merge(df4, how='left', on='comment')

In [8]:
cols = merged.columns.values
cols[3] = 'label_3'
cols[4] = 'label_4'
merged.columns = cols

In [9]:
merged.drop_duplicates(subset ="comment", keep = False, inplace = True)
merged = merged.dropna()
merged = merged.reset_index(drop=True)

In [10]:
hits = np.zeros((merged.shape[0],6))

for i in range(merged.shape[0]):
    if merged.loc[i]['label_1'] == merged.loc[i]['label_2']:
        hits[i][0] += 1
    if merged.loc[i]['label_1'] == merged.loc[i]['label_3']:
        hits[i][1] += 1
    if merged.loc[i]['label_1'] == merged.loc[i]['label_4']:
        hits[i][2] += 1
    if merged.loc[i]['label_2'] == merged.loc[i]['label_3']:
        hits[i][3] += 1
    if merged.loc[i]['label_2'] == merged.loc[i]['label_4']:
        hits[i][4] += 1
    if merged.loc[i]['label_3'] == merged.loc[i]['label_4']:
        hits[i][5] += 1

In [11]:
all_same = 0
three_same_one_diff = 0
two_same_two_diff = 0
two_same_one_diff_one_diff = 0
all_diff = 0

for i in range(hits.shape[0]):
    if hits[i].sum() == 6:
        all_same += 1 
    if hits[i].sum() == 3:
        three_same_one_diff += 1
    if hits[i].sum() == 2:
        two_same_two_diff += 1
        
    if hits[i].sum() == 1:
        two_same_one_diff_one_diff += 1
    if hits[i].sum() == 0:
        all_diff += 1

In [12]:
print('All same: ', all_same)
print('Three same, one different: ', three_same_one_diff)
print('Two same, two different (same to one another): ', two_same_two_diff)
print('Two same, two different (different from one another): ', two_same_one_diff_one_diff)
print('All different: ', all_diff)
print('Total comments: ', merged.shape[0])

All same:  354
Three same, one different:  95
Two same, two different (same to one another):  12
Two same, two different (different from one another):  14
All different:  1
Total comments:  476


In [13]:
print('All same (%): ', round(all_same/merged.shape[0]*100))
print('Three same, one different (%): ',  round(three_same_one_diff/merged.shape[0]*100))
print('Two same, two different (same to one another) (%): ',  round(two_same_two_diff/merged.shape[0]*100))
print('Two same, two different (different from one another) (%): ',  round(two_same_one_diff_one_diff/merged.shape[0]*100))
print('All different (%): ',  round(all_diff/merged.shape[0]*100))

All same (%):  74
Three same, one different (%):  20
Two same, two different (same to one another) (%):  3
Two same, two different (different from one another) (%):  3
All different (%):  0


In [14]:
print('Procenat podudarnosti izmedju Laze i Matije: ', round(hits[:,0].sum()/merged.shape[0]*100))
print('Procenat podudarnosti izmedju Laze i Mateje: ', round(hits[:,1].sum()/merged.shape[0]*100))
print('Procenat podudarnosti izmedju Laze i Munje: ', round(hits[:,2].sum()/merged.shape[0]*100))
print('Procenat podudarnosti izmedju Matije i Mateje: ', round(hits[:,3].sum()/merged.shape[0]*100))
print('Procenat podudarnosti izmedju Matije i Munje: ', round(hits[:,4].sum()/merged.shape[0]*100))
print('Procenat podudarnosti izmedju Mateje i Munje: ', round(hits[:,5].sum()/merged.shape[0]*100))

Procenat podudarnosti izmedju Laze i Matije:  87.0
Procenat podudarnosti izmedju Laze i Mateje:  93.0
Procenat podudarnosti izmedju Laze i Munje:  83.0
Procenat podudarnosti izmedju Matije i Mateje:  89.0
Procenat podudarnosti izmedju Matije i Munje:  78.0
Procenat podudarnosti izmedju Mateje i Munje:  83.0


## Priprema common podataka za dodavanje celokupnim podacima

In [15]:
cols = ['NaturalLanguageID', 'ProgrammingLanguageName', 'RepoID', 'SourceID', 'CommentID', 'comment', 'label']
df = pd.read_csv('Data\\Common\OutDataCommon1.txt', sep='\t', lineterminator='\r', header=None)
df.columns = cols

In [16]:
for i in range(df.shape[0]):
    if df['NaturalLanguageID'][i][0] == '\n':
        df['NaturalLanguageID'][i] = df['NaturalLanguageID'][i][1:]

In [17]:
df

,NaturalLanguageID,ProgrammingLanguageName,RepoID,SourceID,CommentID,comment,label
0,SR,Java,BILD-IT-Advanced-master,src\grupniProjekat_HotelManagement\Hotel.java,BILD-IT-Advanced-master/src\grupniProjekat_Hot...,klik na dugme guest na pocetku programa \n,Functional-Inline
1,SR,Java,BILD-IT-Advanced-master,src\grupniProjekat_HotelManagement\Hotel.java,BILD-IT-Advanced-master/src\grupniProjekat_Hot...,prikaz slobodnih soba za izabrani tip priliko...,Functional-Inline
2,SR,Java,BILD-IT-Advanced-master,src\grupniProjekat_HotelManagement\Hotel.java,BILD-IT-Advanced-master/src\grupniProjekat_Hot...,gost notifikuje adminu da zeli check out iz h...,Functional-Inline
3,SR,Java,BILD-IT-Advanced-master,src\grupniProjekat_HotelManagement\MyConnector...,BILD-IT-Advanced-master/src\grupniProjekat_Hot...,\n * Sefer Kuduzoviæ \n,General
4,SR,Java,BILD-IT-Advanced-master,src\miniProjekat_05_02_2016\BattleShip.java,BILD-IT-Advanced-master/src\miniProjekat_05_02...,pravimo board velicine 5 * 5 \n,Functional-Inline
...,...,...,...,...,...,...,...
587,SR,Java,SBZ-master,drools-spring-v2-app\src\main\java\drools\spri...,SBZ-master/drools-spring-v2-app\src\main\java\...,racun.setOriginalnaUkupnaCena(racunDTO.getOrig...,Code
588,SR,Java,SCStemmers-master,src\weka\core\stemmers\MilosevicStemmer.java,SCStemmers-master/src\weka\core\stemmers\Milos...,\n * Rečnik koji se koristi za normalizovanje...,Functional-Inline
589,SR,Java,SCStemmers-master,src\weka\core\stemmers\MilosevicStemmer.java,SCStemmers-master/src\weka\core\stemmers\Milos...,Očuvava kapitalizaciju na početku rečenice \n...,Functional-Inline
590,SR,Java,SCStemmers-master,src\weka\core\stemmers\MilosevicStemmer.java,SCStemmers-master/src\weka\core\stemmers\Milos...,\n * Ako se radi o karakteru koje spada u eng...,Functional-Inline


In [18]:
for i in range(merged.shape[0]):
    # dodeljuje se najverovatnija vrednost zajednickim komentarima
    if merged['comment'][i] in df.comment.values:
        arr = np.zeros(4) # niz u koji ce se upisati koliko se koji label pojavljuje
        for j in range(4):
            # za prvi label se prebroji koliko puta se pojavio i medju ostalim label-ima, i tako dalje za 2, 3, 4.
            arr[j] = merged.loc[i].str.count(merged.loc[i]['label_{}'.format(j+1)]).sum()
        # index u df-u gde je komentar jednak komentaru koji razmatramo u merged tabeli
        idx = df.loc[df['comment'] == merged['comment'][i]].index.values[0] 
        # za prethodno pronadjen red u data frame-u dodljuje se najcesci label
        df.loc[idx]['label'] = merged['label_{}'.format(arr.argmax()+1)][i]

In [19]:
df_common = pd.DataFrame(columns=df.columns)

# zadrzavaju se samo komentari iz df koji se nalaze i u merged 
for i in range(df.shape[0]):
    if df['comment'][i] in merged.comment.values:
        df_common = df_common.append(df.loc[i])

In [20]:
df_common = df_common.reset_index(drop=True)

In [21]:
df_common

,NaturalLanguageID,ProgrammingLanguageName,RepoID,SourceID,CommentID,comment,label
0,SR,Java,BILD-IT-Advanced-master,src\grupniProjekat_HotelManagement\Hotel.java,BILD-IT-Advanced-master/src\grupniProjekat_Hot...,klik na dugme guest na pocetku programa \n,Functional-Inline
1,SR,Java,BILD-IT-Advanced-master,src\grupniProjekat_HotelManagement\Hotel.java,BILD-IT-Advanced-master/src\grupniProjekat_Hot...,prikaz slobodnih soba za izabrani tip priliko...,Functional-Inline
2,SR,Java,BILD-IT-Advanced-master,src\grupniProjekat_HotelManagement\Hotel.java,BILD-IT-Advanced-master/src\grupniProjekat_Hot...,gost notifikuje adminu da zeli check out iz h...,Functional-Inline
3,SR,Java,BILD-IT-Advanced-master,src\grupniProjekat_HotelManagement\MyConnector...,BILD-IT-Advanced-master/src\grupniProjekat_Hot...,\n * Sefer Kuduzoviæ \n,General
4,SR,Java,BILD-IT-Advanced-master,src\miniProjekat_05_02_2016\BattleShip.java,BILD-IT-Advanced-master/src\miniProjekat_05_02...,pravimo board velicine 5 * 5 \n,Functional-Inline
...,...,...,...,...,...,...,...
471,SR,Java,SBZ-master,drools-spring-v2-app\src\main\java\drools\spri...,SBZ-master/drools-spring-v2-app\src\main\java\...,racun.setOriginalnaUkupnaCena(racunDTO.getOrig...,Code
472,SR,Java,SCStemmers-master,src\weka\core\stemmers\MilosevicStemmer.java,SCStemmers-master/src\weka\core\stemmers\Milos...,\n * Rečnik koji se koristi za normalizovanje...,Functional-Inline
473,SR,Java,SCStemmers-master,src\weka\core\stemmers\MilosevicStemmer.java,SCStemmers-master/src\weka\core\stemmers\Milos...,Očuvava kapitalizaciju na početku rečenice \n...,Functional-Inline
474,SR,Java,SCStemmers-master,src\weka\core\stemmers\MilosevicStemmer.java,SCStemmers-master/src\weka\core\stemmers\Milos...,\n * Ako se radi o karakteru koje spada u eng...,Functional-Inline


## Ucitavanje svih podataka

In [22]:
def create_df():
    for id_ in range(1,5):
        try:
            cols = ['NaturalLanguageID', 'ProgrammingLanguageName', 'RepoID', 'SourceID', 'CommentID', 'comment', 'label']
            df = pd.read_csv('Data\\All data\\OutData{}.txt'.format(id_), sep='\t', lineterminator='\r', header=None)
            df.columns = cols
            #df = df.drop(['Programming language', 'project', 'source', 'line'],axis=1)
            if id_ == 1:
                dfs = df
            else:
                dfs = pd.concat([dfs,df], ignore_index=True)
        except:
            with open('Data\\All data\\OutData{}.txt'.format(id_), "r", encoding="utf-8",  newline="\r\n") as source:
                for line in source:
                    line = line.replace("\n", "")
                    part = line.split("\t")
                    if len(part) != 7:
                        print('POPRAVITI .TXT')
                        print(part)
    return dfs

In [23]:
df = create_df()

In [24]:
df = pd.concat([df, df_common], ignore_index=True)

In [25]:
df = df.dropna()

In [26]:
df

,NaturalLanguageID,ProgrammingLanguageName,RepoID,SourceID,CommentID,comment,label
0,SR,Java,BILD-IT-Advanced-master,src\grupniProjekat_02_03_2016\SearchEngine.java,BILD-IT-Advanced-master/src\grupniProjekat_02_...,Ispisujemo poruku da je drajver loadovan \n,Functional-Inline
1,\nSR,Java,BILD-IT-Advanced-master,src\grupniProjekat_02_03_2016\SearchEngine.java,BILD-IT-Advanced-master/src\grupniProjekat_02_...,Ispis država za uneseni kontinent \n,Functional-Inline
2,\nSR,Java,BILD-IT-Advanced-master,src\grupniProjekat_02_03_2016\SearchEngine.java,BILD-IT-Advanced-master/src\grupniProjekat_02_...,Slanje upute \n,Functional-Inline
3,\nSR,Java,BILD-IT-Advanced-master,src\grupniProjekat_02_03_2016\SearchEngine.java,BILD-IT-Advanced-master/src\grupniProjekat_02_...,Ispis gradova koji pocinju odreðenim slovom \n,Functional-Inline
4,\nSR,Java,BILD-IT-Advanced-master,src\grupniProjekat_02_03_2016\SearchEngine.java,BILD-IT-Advanced-master/src\grupniProjekat_02_...,Hvatalica grešaka \n,Functional-Inline
...,...,...,...,...,...,...,...
6360,SR,Java,SBZ-master,drools-spring-v2-app\src\main\java\drools\spri...,SBZ-master/drools-spring-v2-app\src\main\java\...,racun.setOriginalnaUkupnaCena(racunDTO.getOrig...,Code
6361,SR,Java,SCStemmers-master,src\weka\core\stemmers\MilosevicStemmer.java,SCStemmers-master/src\weka\core\stemmers\Milos...,\n * Rečnik koji se koristi za normalizovanje...,Functional-Inline
6362,SR,Java,SCStemmers-master,src\weka\core\stemmers\MilosevicStemmer.java,SCStemmers-master/src\weka\core\stemmers\Milos...,Očuvava kapitalizaciju na početku rečenice \n...,Functional-Inline
6363,SR,Java,SCStemmers-master,src\weka\core\stemmers\MilosevicStemmer.java,SCStemmers-master/src\weka\core\stemmers\Milos...,\n * Ako se radi o karakteru koje spada u eng...,Functional-Inline


In [27]:
for i in range(df.shape[0]):
    if df['NaturalLanguageID'][i][0] == '\n':
        df['NaturalLanguageID'][i] = df['NaturalLanguageID'][i][1:]

In [28]:
df.loc[df['NaturalLanguageID'] == 'UNDEF']

,NaturalLanguageID,ProgrammingLanguageName,RepoID,SourceID,CommentID,comment,label
766,UNDEF,Java,dusankrivosija-som-13d92a2b4d2a,DCT\src\dct\DCT.java,dusankrivosija-som-13d92a2b4d2a/DCT\src\dct\DC...,"2,0",Functional-Method
767,UNDEF,Java,dusankrivosija-som-13d92a2b4d2a,DCT\src\dct\DCT.java,dusankrivosija-som-13d92a2b4d2a/DCT\src\dct\DC...,"1,2",Functional-Method
768,UNDEF,Java,dusankrivosija-som-13d92a2b4d2a,DCT\src\dct\DCT.java,dusankrivosija-som-13d92a2b4d2a/DCT\src\dct\DC...,"3,1",Functional-Method
769,UNDEF,Java,dusankrivosija-som-13d92a2b4d2a,DCT\src\dct\DCT.java,dusankrivosija-som-13d92a2b4d2a/DCT\src\dct\DC...,"0,5",Functional-Inline
770,UNDEF,Java,dusankrivosija-som-13d92a2b4d2a,DCT\src\dct\DCT.java,dusankrivosija-som-13d92a2b4d2a/DCT\src\dct\DC...,"4,1",Functional-Inline
...,...,...,...,...,...,...,...
5543,UNDEF,Java,OOP-master,predavanja\primeri-java\src\P040502Promenljive...,OOP-master/predavanja\primeri-java\src\P040502...,5,Functional-Inline
5544,UNDEF,Java,OOP-master,predavanja\primeri-java\src\P040502Promenljive...,OOP-master/predavanja\primeri-java\src\P040502...,10,Functional-Inline
5568,UNDEF,Java,OOP-master,predavanja\primeri-java\src\P060500Dvodimenzio...,OOP-master/predavanja\primeri-java\src\P060500...,15 30,Functional-Inline
5794,UNDEF,Java,OOP-master,vezbe\primeri-java\09.kolekcije\src\dodatnoTok...,OOP-master/vezbe\primeri-java\09.kolekcije\src...,3,Functional-Inline


In [29]:
df = df.drop(df.loc[df['NaturalLanguageID'] == 'UNDEF'].index)

In [30]:
df

,NaturalLanguageID,ProgrammingLanguageName,RepoID,SourceID,CommentID,comment,label
0,SR,Java,BILD-IT-Advanced-master,src\grupniProjekat_02_03_2016\SearchEngine.java,BILD-IT-Advanced-master/src\grupniProjekat_02_...,Ispisujemo poruku da je drajver loadovan \n,Functional-Inline
1,SR,Java,BILD-IT-Advanced-master,src\grupniProjekat_02_03_2016\SearchEngine.java,BILD-IT-Advanced-master/src\grupniProjekat_02_...,Ispis država za uneseni kontinent \n,Functional-Inline
2,SR,Java,BILD-IT-Advanced-master,src\grupniProjekat_02_03_2016\SearchEngine.java,BILD-IT-Advanced-master/src\grupniProjekat_02_...,Slanje upute \n,Functional-Inline
3,SR,Java,BILD-IT-Advanced-master,src\grupniProjekat_02_03_2016\SearchEngine.java,BILD-IT-Advanced-master/src\grupniProjekat_02_...,Ispis gradova koji pocinju odreðenim slovom \n,Functional-Inline
4,SR,Java,BILD-IT-Advanced-master,src\grupniProjekat_02_03_2016\SearchEngine.java,BILD-IT-Advanced-master/src\grupniProjekat_02_...,Hvatalica grešaka \n,Functional-Inline
...,...,...,...,...,...,...,...
6360,SR,Java,SBZ-master,drools-spring-v2-app\src\main\java\drools\spri...,SBZ-master/drools-spring-v2-app\src\main\java\...,racun.setOriginalnaUkupnaCena(racunDTO.getOrig...,Code
6361,SR,Java,SCStemmers-master,src\weka\core\stemmers\MilosevicStemmer.java,SCStemmers-master/src\weka\core\stemmers\Milos...,\n * Rečnik koji se koristi za normalizovanje...,Functional-Inline
6362,SR,Java,SCStemmers-master,src\weka\core\stemmers\MilosevicStemmer.java,SCStemmers-master/src\weka\core\stemmers\Milos...,Očuvava kapitalizaciju na početku rečenice \n...,Functional-Inline
6363,SR,Java,SCStemmers-master,src\weka\core\stemmers\MilosevicStemmer.java,SCStemmers-master/src\weka\core\stemmers\Milos...,\n * Ako se radi o karakteru koje spada u eng...,Functional-Inline


In [31]:
df = df.reset_index(drop=True)

In [32]:
df

,NaturalLanguageID,ProgrammingLanguageName,RepoID,SourceID,CommentID,comment,label
0,SR,Java,BILD-IT-Advanced-master,src\grupniProjekat_02_03_2016\SearchEngine.java,BILD-IT-Advanced-master/src\grupniProjekat_02_...,Ispisujemo poruku da je drajver loadovan \n,Functional-Inline
1,SR,Java,BILD-IT-Advanced-master,src\grupniProjekat_02_03_2016\SearchEngine.java,BILD-IT-Advanced-master/src\grupniProjekat_02_...,Ispis država za uneseni kontinent \n,Functional-Inline
2,SR,Java,BILD-IT-Advanced-master,src\grupniProjekat_02_03_2016\SearchEngine.java,BILD-IT-Advanced-master/src\grupniProjekat_02_...,Slanje upute \n,Functional-Inline
3,SR,Java,BILD-IT-Advanced-master,src\grupniProjekat_02_03_2016\SearchEngine.java,BILD-IT-Advanced-master/src\grupniProjekat_02_...,Ispis gradova koji pocinju odreðenim slovom \n,Functional-Inline
4,SR,Java,BILD-IT-Advanced-master,src\grupniProjekat_02_03_2016\SearchEngine.java,BILD-IT-Advanced-master/src\grupniProjekat_02_...,Hvatalica grešaka \n,Functional-Inline
...,...,...,...,...,...,...,...
6262,SR,Java,SBZ-master,drools-spring-v2-app\src\main\java\drools\spri...,SBZ-master/drools-spring-v2-app\src\main\java\...,racun.setOriginalnaUkupnaCena(racunDTO.getOrig...,Code
6263,SR,Java,SCStemmers-master,src\weka\core\stemmers\MilosevicStemmer.java,SCStemmers-master/src\weka\core\stemmers\Milos...,\n * Rečnik koji se koristi za normalizovanje...,Functional-Inline
6264,SR,Java,SCStemmers-master,src\weka\core\stemmers\MilosevicStemmer.java,SCStemmers-master/src\weka\core\stemmers\Milos...,Očuvava kapitalizaciju na početku rečenice \n...,Functional-Inline
6265,SR,Java,SCStemmers-master,src\weka\core\stemmers\MilosevicStemmer.java,SCStemmers-master/src\weka\core\stemmers\Milos...,\n * Ako se radi o karakteru koje spada u eng...,Functional-Inline


In [33]:
df.loc[df['NaturalLanguageID'] == 'EN/SR']

,NaturalLanguageID,ProgrammingLanguageName,RepoID,SourceID,CommentID,comment,label
33,EN/SR,Java,BILD-IT-Advanced-master,src\miniProjekat_14_01_2016\Kalendar.java,BILD-IT-Advanced-master/src\miniProjekat_14_01...,\n * Author: Sefer Kuduzovic \n,General
132,EN/SR,Java,BILD-IT-Advanced-master,src\zadaca_15_01_2016\Zadatak4.java,BILD-IT-Advanced-master/src\zadaca_15_01_2016\...,\n * Author: Sefer Kuduzovic \n,General
146,EN/SR,Java,BILD-IT-Advanced-master,src\zadaca_16_01_2016\Zadatak4.java,BILD-IT-Advanced-master/src\zadaca_16_01_2016\...,\n * Author: Sefer Kuduzovic \n,General
147,EN/SR,Java,BILD-IT-Advanced-master,src\zadaca_16_01_2016\Zadatak5.java,BILD-IT-Advanced-master/src\zadaca_16_01_2016\...,\n * Author: Sefer Kuduzovic \n,General
258,EN/SR,Java,delboy-exgerm-b858e2a4b6f5,Core\tk\exgerm\core\actions\ActionRegister.java,delboy-exgerm-b858e2a4b6f5/Core\tk\exgerm\core...,\n * Čuva sve instance akcija koje pravi Core...,Functional-Module
...,...,...,...,...,...,...,...
6055,EN/SR,Java,delboy-sportskakladionica-web-57e11bf14101,src\rs\in\delic\wp\Client\User.java,delboy-sportskakladionica-web-57e11bf14101/src...,\n * @author del-boy \n * @version 0.01 \n * ...,General
6059,EN/SR,Java,delboy-sportskakladionica-web-57e11bf14101,src\rs\in\delic\wp\Client\Commands\ListFinishe...,delboy-sportskakladionica-web-57e11bf14101/src...,\n * @author del-boy \n * \n,General
6073,EN/SR,Java,delboy-sportskakladionica-web-57e11bf14101,src\rs\in\delic\wp\Server\Commands\AddBetTicke...,delboy-sportskakladionica-web-57e11bf14101/src...,\n * @author del-boy \n * \n * Klasa koja vrs...,Functional-Module
6076,EN/SR,Java,delboy-sportskakladionica-web-57e11bf14101,src\rs\in\delic\wp\Server\Commands\MatchResult...,delboy-sportskakladionica-web-57e11bf14101/src...,\n * @author del-boy \n * \n * Radi komunikac...,Functional-Module


In [34]:
droped_duplicates = df.drop_duplicates(['comment'])

In [35]:
df.shape

(6267, 7)

In [36]:
droped_duplicates.shape

(4750, 7)

In [37]:
# This is for the case we want to use Functional for all comments

for i in range(df.shape[0]):
    df.loc[i]['label'] = re.sub('-Inline', '', df['label'][i])
    df.loc[i]['label'] = re.sub('-Method', '', df['label'][i])
    df.loc[i]['label'] = re.sub('-Module', '', df['label'][i])


In [38]:
df

,NaturalLanguageID,ProgrammingLanguageName,RepoID,SourceID,CommentID,comment,label
0,SR,Java,BILD-IT-Advanced-master,src\grupniProjekat_02_03_2016\SearchEngine.java,BILD-IT-Advanced-master/src\grupniProjekat_02_...,Ispisujemo poruku da je drajver loadovan \n,Functional
1,SR,Java,BILD-IT-Advanced-master,src\grupniProjekat_02_03_2016\SearchEngine.java,BILD-IT-Advanced-master/src\grupniProjekat_02_...,Ispis država za uneseni kontinent \n,Functional
2,SR,Java,BILD-IT-Advanced-master,src\grupniProjekat_02_03_2016\SearchEngine.java,BILD-IT-Advanced-master/src\grupniProjekat_02_...,Slanje upute \n,Functional
3,SR,Java,BILD-IT-Advanced-master,src\grupniProjekat_02_03_2016\SearchEngine.java,BILD-IT-Advanced-master/src\grupniProjekat_02_...,Ispis gradova koji pocinju odreðenim slovom \n,Functional
4,SR,Java,BILD-IT-Advanced-master,src\grupniProjekat_02_03_2016\SearchEngine.java,BILD-IT-Advanced-master/src\grupniProjekat_02_...,Hvatalica grešaka \n,Functional
...,...,...,...,...,...,...,...
6262,SR,Java,SBZ-master,drools-spring-v2-app\src\main\java\drools\spri...,SBZ-master/drools-spring-v2-app\src\main\java\...,racun.setOriginalnaUkupnaCena(racunDTO.getOrig...,Code
6263,SR,Java,SCStemmers-master,src\weka\core\stemmers\MilosevicStemmer.java,SCStemmers-master/src\weka\core\stemmers\Milos...,\n * Rečnik koji se koristi za normalizovanje...,Functional
6264,SR,Java,SCStemmers-master,src\weka\core\stemmers\MilosevicStemmer.java,SCStemmers-master/src\weka\core\stemmers\Milos...,Očuvava kapitalizaciju na početku rečenice \n...,Functional
6265,SR,Java,SCStemmers-master,src\weka\core\stemmers\MilosevicStemmer.java,SCStemmers-master/src\weka\core\stemmers\Milos...,\n * Ako se radi o karakteru koje spada u eng...,Functional


In [39]:
pd.set_option('display.max_rows', 10)

In [40]:
df.loc[df.duplicated('comment', keep=False)].sort_values('comment')

,NaturalLanguageID,ProgrammingLanguageName,RepoID,SourceID,CommentID,comment,label
5738,SR,Java,raspored-master,src\org\svetovid\raspored\cmd\Main.java,raspored-master/src\org\svetovid\raspored\cmd\...,"""nikad"" ""po-potrebi"" ""uvek"" \n",Functional
1369,SR,Java,raspored-master,src\org\svetovid\raspored\cmd\Main.java,raspored-master/src\org\svetovid\raspored\cmd\...,"""nikad"" ""po-potrebi"" ""uvek"" \n",Functional
858,SR,Java,Language-processor-master,Language-processor\src\hr\unizg\fer\lab3\Izraz...,Language-processor-master/Language-processor\s...,++ i -- \n,Functional
3927,SR,Java,Language-processor-master,Language-processor\src\hr\unizg\fer\labComplet...,Language-processor-master/Language-processor\s...,++ i -- \n,Code
5293,SR,Java,Language-processor-master,Language-processor\src\hr\unizg\fer\lab4\Izraz...,Language-processor-master/Language-processor\s...,++ i -- \n,Functional
...,...,...,...,...,...,...,...
1808,SR,Java,delboy-exgerm-b858e2a4b6f5,GraphTree\tk\exgerm\graphtree\listeners\NodeRe...,delboy-exgerm-b858e2a4b6f5/GraphTree\tk\exgerm...,uzimamo root graf i u zavisnosti od broja nivo...,Functional
4947,SR,Java,delboy-germ-fab96483261f,germ\gui\workspace\WorkspaceTree.java,delboy-germ-fab96483261f/germ\gui\workspace\Wo...,uzimanje popup menu \n,Functional
3480,SR,Java,delboy-germ-fab96483261f,germ\gui\workspace\WorkspaceTree.java,delboy-germ-fab96483261f/germ\gui\workspace\Wo...,uzimanje popup menu \n,Functional
4948,SR,Java,delboy-germ-fab96483261f,germ\gui\workspace\WorkspaceTree.java,delboy-germ-fab96483261f/germ\gui\workspace\Wo...,uzimanje popup menu \n,Functional


In [41]:
droped_duplicates[droped_duplicates.index == 3676]

,NaturalLanguageID,ProgrammingLanguageName,RepoID,SourceID,CommentID,comment,label
3676,SR,Java,dusankrivosija-som-13d92a2b4d2a,Neuroph\org\neuroph\contrib\IACNetwork.java,dusankrivosija-som-13d92a2b4d2a/Neuroph\org\ne...,Burglar,Functional-Inline


In [42]:
df = df.drop_duplicates(['comment'])
df = df.reset_index(drop=True)

In [43]:
df = df.drop(df[df['comment'].str.contains('NON')].index)

In [44]:
df['label'].value_counts()

Functional    4375
Notice         146
Code           110
General         66
ToDo            31
IDE             20
Name: label, dtype: int64

In [45]:
df[df['label'] == 'Notice']

,NaturalLanguageID,ProgrammingLanguageName,RepoID,SourceID,CommentID,comment,label
219,SR,Java,BILD-IT-Advanced-master,src\zadaca_30_01_2016\Zadatak3.java,BILD-IT-Advanced-master/src\zadaca_30_01_2016\...,\n * http://stackoverflow.com/questions/40874...,Notice
332,SR,Java,delboy-exgerm-b858e2a4b6f5,Core\tk\exgerm\core\register\GraphRegisterProx...,delboy-exgerm-b858e2a4b6f5/Core\tk\exgerm\core...,"pošto smo prvo proverili da li graf postoji, ...",Notice
985,SR,Java,OOP-master,predavanja\primeri-java\src\rs\math\oop1\z0705...,OOP-master/predavanja\primeri-java\src\rs\math...,ovo nikad ne treba da bude izvrseno,Notice
1032,SR,Java,OOP-master,predavanja\primeri-java\src\rs\math\oop1\z1001...,OOP-master/predavanja\primeri-java\src\rs\math...,OVAKO NE TREBA RADITI! \n,Notice
1044,SR,Java,OOP-master,predavanja\primeri-java\src\rs\math\oop1\z1903...,OOP-master/predavanja\primeri-java\src\rs\math...,Metod Integer.parseInt() moze dovesti do izba...,Notice
...,...,...,...,...,...,...,...
4306,SR,Java,OOP-master,vezbe\primeri-java\08.genericke.klase\src\prim...,OOP-master/vezbe\primeri-java\08.genericke.kla...,Cesto je korisno imati staticke funkcije koje...,Notice
4313,SR,Java,OOP-master,vezbe\primeri-java\09.kolekcije\src\primer03Se...,OOP-master/vezbe\primeri-java\09.kolekcije\src...,---------------------------------------------...,Notice
4317,SR,Java,OOP-master,vezbe\primeri-java\10.poredjenje.objekata\src\...,OOP-master/vezbe\primeri-java\10.poredjenje.ob...,\n Sortiranje mozemo izvrsiti i nad nasim kla...,Notice
4375,SR,Java,SCStemmers-master,src\weka\core\stemmers\MilosevicStemmer.java,SCStemmers-master/src\weka\core\stemmers\Milos...,Ovi oblici se ne javljaju u ArXiv radu. Oni s...,Notice


In [46]:
df[df['label'] == 'General']

,NaturalLanguageID,ProgrammingLanguageName,RepoID,SourceID,CommentID,comment,label
32,EN/SR,Java,BILD-IT-Advanced-master,src\miniProjekat_14_01_2016\Kalendar.java,BILD-IT-Advanced-master/src\miniProjekat_14_01...,\n * Author: Sefer Kuduzovic \n,General
254,EN/SR,Java,delboy-exgerm-b858e2a4b6f5,Core\tk\exgerm\core\exception\ExGEdgeDoesNotEx...,delboy-exgerm-b858e2a4b6f5/Core\tk\exgerm\core...,\n * @author Tim 2 \n * \n,General
641,SR,Java,delboy-sportskakladionica-mbs2-9fe0a4caa6be,SportBetting\src\rs\in\delic\mbs2\SportBetting...,delboy-sportskakladionica-mbs2-9fe0a4caa6be/Sp...,"\n * Bojan Delić, e11510 \n * Projekat iz pre...",General
642,SR,Java,delboy-sportskakladionica-mbs2-9fe0a4caa6be,SportBetting\src\rs\in\delic\mbs2\SportBetting...,delboy-sportskakladionica-mbs2-9fe0a4caa6be/Sp...,"\n * Bojan Delić, e11510 \n * Projekat iz pre...",General
644,SR,Java,delboy-sportskakladionica-mbs2-9fe0a4caa6be,SportBetting\src\rs\in\delic\mbs2\SportBetting...,delboy-sportskakladionica-mbs2-9fe0a4caa6be/Sp...,"\n * Bojan Delić, e11510 \n * Projekat iz pre...",General
...,...,...,...,...,...,...,...
4219,SR,Java,OOP-master,predavanja\primeri-java\src\rs\math\oop1\z0804...,OOP-master/predavanja\primeri-java\src\rs\math...,\n * \n * Program ilustruje koncept polimorfi...,General
4232,EN/SR,Java,OOP-master,predavanja\primeri-java\src\rs\math\oop1\z1701...,OOP-master/predavanja\primeri-java\src\rs\math...,\n * @version 1.10 17 Aug 1998 \n * @author C...,General
4272,SR,Java,OOP-master,vezbe\primeri-java\02.nizovi.matrice.stringovi...,OOP-master/vezbe\primeri-java\02.nizovi.matric...,/ Program ilistruje kako mozemo sortirati Stri...,General
4383,SR,Java,BILD-IT-Advanced-master,src\grupniProjekat_HotelManagement\MyConnector...,BILD-IT-Advanced-master/src\grupniProjekat_Hot...,\n * Sefer Kuduzoviæ \n,General


In [47]:
df.loc[df.index == 2044]['comment'][2044]

' Prvi nacin, jedna linija :) \\n  igre.sort(Comparator.comparing(VideoIgra::getIme)); \\n '

In [48]:
df = df.drop(df[df.comment.apply(lambda x: len(str(x)) < 6)].index)

In [49]:
df

,NaturalLanguageID,ProgrammingLanguageName,RepoID,SourceID,CommentID,comment,label
0,SR,Java,BILD-IT-Advanced-master,src\grupniProjekat_02_03_2016\SearchEngine.java,BILD-IT-Advanced-master/src\grupniProjekat_02_...,Ispisujemo poruku da je drajver loadovan \n,Functional
1,SR,Java,BILD-IT-Advanced-master,src\grupniProjekat_02_03_2016\SearchEngine.java,BILD-IT-Advanced-master/src\grupniProjekat_02_...,Ispis država za uneseni kontinent \n,Functional
2,SR,Java,BILD-IT-Advanced-master,src\grupniProjekat_02_03_2016\SearchEngine.java,BILD-IT-Advanced-master/src\grupniProjekat_02_...,Slanje upute \n,Functional
3,SR,Java,BILD-IT-Advanced-master,src\grupniProjekat_02_03_2016\SearchEngine.java,BILD-IT-Advanced-master/src\grupniProjekat_02_...,Ispis gradova koji pocinju odreðenim slovom \n,Functional
4,SR,Java,BILD-IT-Advanced-master,src\grupniProjekat_02_03_2016\SearchEngine.java,BILD-IT-Advanced-master/src\grupniProjekat_02_...,Hvatalica grešaka \n,Functional
...,...,...,...,...,...,...,...
4745,SR,Java,RAUM-master,funkcija.java,RAUM-master/funkcija.java/294,sad je podfunkcija oblika funkcija(broj),Functional
4746,SR,Java,SBZ-master,drools-spring-v2-app\src\main\java\drools\spri...,SBZ-master/drools-spring-v2-app\src\main\java\...,racun.setOriginalnaUkupnaCena(racunDTO.getOrig...,Code
4747,SR,Java,SCStemmers-master,src\weka\core\stemmers\MilosevicStemmer.java,SCStemmers-master/src\weka\core\stemmers\Milos...,\n * Rečnik koji se koristi za normalizovanje...,Functional
4748,SR,Java,SCStemmers-master,src\weka\core\stemmers\MilosevicStemmer.java,SCStemmers-master/src\weka\core\stemmers\Milos...,Očuvava kapitalizaciju na početku rečenice \n...,Functional


In [50]:
df = df.reset_index(drop=True)

In [51]:
df

,NaturalLanguageID,ProgrammingLanguageName,RepoID,SourceID,CommentID,comment,label
0,SR,Java,BILD-IT-Advanced-master,src\grupniProjekat_02_03_2016\SearchEngine.java,BILD-IT-Advanced-master/src\grupniProjekat_02_...,Ispisujemo poruku da je drajver loadovan \n,Functional
1,SR,Java,BILD-IT-Advanced-master,src\grupniProjekat_02_03_2016\SearchEngine.java,BILD-IT-Advanced-master/src\grupniProjekat_02_...,Ispis država za uneseni kontinent \n,Functional
2,SR,Java,BILD-IT-Advanced-master,src\grupniProjekat_02_03_2016\SearchEngine.java,BILD-IT-Advanced-master/src\grupniProjekat_02_...,Slanje upute \n,Functional
3,SR,Java,BILD-IT-Advanced-master,src\grupniProjekat_02_03_2016\SearchEngine.java,BILD-IT-Advanced-master/src\grupniProjekat_02_...,Ispis gradova koji pocinju odreðenim slovom \n,Functional
4,SR,Java,BILD-IT-Advanced-master,src\grupniProjekat_02_03_2016\SearchEngine.java,BILD-IT-Advanced-master/src\grupniProjekat_02_...,Hvatalica grešaka \n,Functional
...,...,...,...,...,...,...,...
4705,SR,Java,RAUM-master,funkcija.java,RAUM-master/funkcija.java/294,sad je podfunkcija oblika funkcija(broj),Functional
4706,SR,Java,SBZ-master,drools-spring-v2-app\src\main\java\drools\spri...,SBZ-master/drools-spring-v2-app\src\main\java\...,racun.setOriginalnaUkupnaCena(racunDTO.getOrig...,Code
4707,SR,Java,SCStemmers-master,src\weka\core\stemmers\MilosevicStemmer.java,SCStemmers-master/src\weka\core\stemmers\Milos...,\n * Rečnik koji se koristi za normalizovanje...,Functional
4708,SR,Java,SCStemmers-master,src\weka\core\stemmers\MilosevicStemmer.java,SCStemmers-master/src\weka\core\stemmers\Milos...,Očuvava kapitalizaciju na početku rečenice \n...,Functional


Preprocessing all comments

In [52]:
for i in range(df.shape[0]):
    # pretvaranje celog teksta u mala slova
    df.loc[i]['comment'] = df.loc[i]['comment'].lower()
    # izbacivanje special character-a
    df['comment'][i] = re.sub(r'\W', ' ', df['comment'][i])
    # izbacivanje new line oznake
    df['comment'][i] = re.sub(r'\b[n]\b', '', df['comment'][i])
    # zamena vise razmaka s jednim razmakom
    df['comment'][i] = re.sub(r'\s+', ' ', df['comment'][i], flags=re.I)

In [53]:
for i in range(30):
    print(df['comment'][i])

 ispisujemo poruku da je drajver loadovan 
 ispis država za uneseni kontinent 
 slanje upute 
 ispis gradova koji pocinju odreðenim slovom 
 hvatalica grešaka 
 konstruktor sa data fields 
 guest button 
 ok button za login admin 
 zatvaranje pregleda racuna 
 button za pregled racuna za datog korisnika 
 button za spasavanje update a gosta 
 panel sa boxom i poljem za unos nove sobe na zahtjev gosta 
 button za prikaz slobodnih soba datog tipa 
 tabs admin halim 
 polja za unos podataka o gostu 
 sljedeæe linije su za panel 1 i panel 2 admin window za ostale usluge hotela 
 prilikom logina ako zelimo da odustanemo i vratimo se na poèetak 
 prikaz notifikacija adminu ukoliko neki gost zeli check out 
 ucitavanje podataka o sobi i gostu za kreiranje racuna za prikaz 
 prijava novog gosta u hotel 
 brisanje notifikacije za check out 
 editovanje podataka o gostu tab2 admin windows guest lookup and change podataka 
 gost sacuva podatke koje je mjenjao direktno u bazu 
 sql driver 
 notifi

In [54]:
df

,NaturalLanguageID,ProgrammingLanguageName,RepoID,SourceID,CommentID,comment,label
0,SR,Java,BILD-IT-Advanced-master,src\grupniProjekat_02_03_2016\SearchEngine.java,BILD-IT-Advanced-master/src\grupniProjekat_02_...,ispisujemo poruku da je drajver loadovan,Functional
1,SR,Java,BILD-IT-Advanced-master,src\grupniProjekat_02_03_2016\SearchEngine.java,BILD-IT-Advanced-master/src\grupniProjekat_02_...,ispis država za uneseni kontinent,Functional
2,SR,Java,BILD-IT-Advanced-master,src\grupniProjekat_02_03_2016\SearchEngine.java,BILD-IT-Advanced-master/src\grupniProjekat_02_...,slanje upute,Functional
3,SR,Java,BILD-IT-Advanced-master,src\grupniProjekat_02_03_2016\SearchEngine.java,BILD-IT-Advanced-master/src\grupniProjekat_02_...,ispis gradova koji pocinju odreðenim slovom,Functional
4,SR,Java,BILD-IT-Advanced-master,src\grupniProjekat_02_03_2016\SearchEngine.java,BILD-IT-Advanced-master/src\grupniProjekat_02_...,hvatalica grešaka,Functional
...,...,...,...,...,...,...,...
4705,SR,Java,RAUM-master,funkcija.java,RAUM-master/funkcija.java/294,sad je podfunkcija oblika funkcija broj,Functional
4706,SR,Java,SBZ-master,drools-spring-v2-app\src\main\java\drools\spri...,SBZ-master/drools-spring-v2-app\src\main\java\...,racun setoriginalnaukupnacena racundto getorig...,Code
4707,SR,Java,SCStemmers-master,src\weka\core\stemmers\MilosevicStemmer.java,SCStemmers-master/src\weka\core\stemmers\Milos...,rečnik koji se koristi za normalizovanje čest...,Functional
4708,SR,Java,SCStemmers-master,src\weka\core\stemmers\MilosevicStemmer.java,SCStemmers-master/src\weka\core\stemmers\Milos...,očuvava kapitalizaciju na početku rečenice pr...,Functional


## ML 

In [55]:
df['comment'].values

array([' ispisujemo poruku da je drajver loadovan ',
       ' ispis država za uneseni kontinent ', ' slanje upute ', ...,
       ' rečnik koji se koristi za normalizovanje često korišćenih nepravilnih glagola p i a dictionary which is used to normalize the frequently used irregular verbs i ',
       ' očuvava kapitalizaciju na početku rečenice preserves the capitalization at the start of a sentence ',
       ' dužina u karakterima najdužeg sufiksnog pravila p i length in characters of the longest suffix rule i '],
      dtype=object)

In [56]:
df.label.value_counts()

Functional    4342
Notice         142
Code           110
General         66
ToDo            30
IDE             20
Name: label, dtype: int64

In [57]:
inline_rnd = df[df.label == 'Functional-Inline'].sample(500)

ValueError: a must be greater than 0 unless no samples are taken

In [ ]:
method_rnd = df[df.label == 'Functional-Method'].sample(500)

In [ ]:
df_proba = pd.concat([df[df.label == 'Functional-Module'], df[df.label == 'Functional-Module'],
                     df[df.label == 'Notice'], df[df.label == 'Code'],
                     df[df.label == 'General'], df[df.label == 'ToDo'],
                     df[df.label == 'IDE'], inline_rnd, method_rnd], ignore_index=True)

In [ ]:
df_proba

In [ ]:
df['label'].values

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(df['comment'].values, df['label'].values, test_size=0.2, random_state=1000)

In [ ]:
type(x_test)

In [ ]:
# Pokrenuti ovo za koriscenje stemmovanja

import nltk
nltk.download('punkt')

import StemmerByNikola
from StemmerByNikola import stem_str
    
stemmed_x_train = []
for doc in x_train:
    stemmed_x_train.append(stem_str(doc))
    
stemmed_x_test = []
for doc in x_test:
    stemmed_x_test.append(stem_str(doc))
    
x_train = np.array(stemmed_x_train)
x_test = np.array(stemmed_x_test)

In [ ]:
x_train

In [ ]:
# Deo gde se radi obicna vektorizacija bez TF-IDF
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(analyzer='word', # lowercase=True, # Menjanjem ovoga testiramo lowercase uticaj
                             ngram_range=(1, 1), # Podesavanjem ovoga testiramo bigramski i trigramski uticaj
                             min_df=1, max_df=1.0) # Podesavanjem ova dva testiracemo uticaj frekvencijskog filtriranja

# vectorizer = CountVectorizer()

vectorizer.fit(x_train)

x_train = vectorizer.transform(x_train)
x_test  = vectorizer.transform(x_test)
x_train

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidfconverter = TfidfTransformer()
X_train = tfidfconverter.fit_transform(x_train).toarray()
X_test = tfidfconverter.fit_transform(x_test).toarray()

## Logistic regression L1 penalty

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

classifier = LogisticRegression(penalty='l1')
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

print("Accuracy:", classifier.score(X_test, y_test))
print("F1 score:", f1_score(y_test, y_pred, average='macro'))

## Logistic regression L2 penalty

In [ ]:
from sklearn.metrics import classification_report

classifier = LogisticRegression(penalty='l2')
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

print("Accuracy:", classifier.score(X_test, y_test))
print("F1 score:", f1_score(y_test, y_pred, average='macro'))

## Grid search for finding the optimal C value

In [ ]:
from sklearn.model_selection import GridSearchCV

#parameters = {'C':[0.1, 0.5, 1, 1.5, 2, 5]}
parameters = {'C':[0.5, 1, 2, 5, 7, 10]}
log_reg = LogisticRegression(penalty='l2')
clf = GridSearchCV(log_reg, parameters, scoring='f1_macro')
clf.fit(X_train, y_train)
#sorted(clf.cv_results_.keys())

In [ ]:
clf.best_params_

In [ ]:
y_true, y_pred = y_test, clf.predict(X_test)
print(classification_report(y_true, y_pred))
print("Accuracy:", clf.score(X_test, y_test))
print("F1 score:", f1_score(y_test, y_pred, average='weighted'))

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred, labels = df['label'].unique())

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sn
%matplotlib inline

def plot_confusion_matrix(y_test, y_pred):
    data = confusion_matrix(y_test, y_pred)
    df_cm = pd.DataFrame(data, columns=np.unique(y_test), index = np.unique(y_test))
    df_cm.index.name = 'Actual'
    df_cm.columns.name = 'Predicted'
    plt.figure(figsize = (10,7))
    sn.set(font_scale=1.4)#for label size
    sn.heatmap(df_cm, cmap="Blues", annot=True, fmt="d", annot_kws={"size": 16})# font size

In [ ]:
plot_confusion_matrix(y_test, y_pred)

## Multinomial Naive Bayes Classifier

In [ ]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print("Accuracy:", clf.score(X_test, y_test))
print("F1 score:", f1_score(y_test, y_pred, average='weighted'))

In [ ]:
plot_confusion_matrix(y_test, y_pred)

## Bernoulli Naive Bayes Classifier

In [ ]:
from sklearn.naive_bayes import BernoulliNB

clf = BernoulliNB()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print("Accuracy:", clf.score(X_test, y_test))
print("F1 score:", f1_score(y_test, y_pred, average='macro'))

In [ ]:
plot_confusion_matrix(y_test, y_pred)

## Support Vector Machine

In [ ]:
from sklearn.svm import LinearSVC

clf = LinearSVC(penalty='l2')
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print("Accuracy:", clf.score(X_test, y_test))
print("F1 score:", f1_score(y_test, y_pred, average='macro'))

In [ ]:
parameters = {'C':[0.5, 1, 2, 5, 7, 10]}
log_reg = LogisticRegression(penalty='l2')
clf = GridSearchCV(log_reg, parameters, scoring='f1_macro')
clf.fit(X_train, y_train)

print('Best C value:', clf.best_params_)

y_true, y_pred = y_test, clf.predict(X_test)
print(classification_report(y_true, y_pred))
print("Accuracy:", clf.score(X_test, y_test))
print("F1 score:", f1_score(y_test, y_pred, average='weighted'))

In [ ]:
plot_confusion_matrix(y_true, y_pred)

In [ ]:
vectorizer.vocabulary_

In [ ]:
for comm in df.loc[df['label'] == 'Notice']['comment']:
    print(comm)
    print('\n')

In [ ]:
df